# Artigo para a Diarização

Este código diversas soluções sobre diarização.

Inicializando com os pacotes e funções que serão utilizadas.

In [111]:
from dotenv import load_dotenv
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
from openai import OpenAI
import os
import json
import re

In [108]:
path = os.path.abspath("")

path = path.replace("\\", "/")

Alterar os datasets aqui

In [101]:
join = '/dataset/08_oitavo'
input_video_path = path + join + '/video.mp4'
output_audio_path = path + join + '/audio_new.mp3' 

In [102]:
#path_split1 = 'C:/Users/anton/Videos/task2/1_half_audio.mp3'
#path_split2 = 'C:/Users/anton/Videos/task2/2_half_audio.mp3'

# Lendo o video 
video_clip = VideoFileClip(input_video_path)

# Obtendo a duracao do video
video_duration = video_clip.duration

# Extraindo o audio para as tarefas
audio_clip = video_clip.audio
audio_clip.write_audiofile(output_audio_path)

MoviePy - Writing audio in c:/Users/anton/Videos/boll/boll/diarizacao_artigo/diarizacao/dataset/08_oitavo/audio_new.mp3


MoviePy - Done.


# Diarização OpenAI

## Funções e Pacotes

In [7]:
load_dotenv()

# Funcao para a trascricao usando Whisper
class OpenAIWhisper():
    def __init__(self):
        self.client = OpenAI(api_key=os.environ.get('OPENAI_KEY'))

    def get_completion_whisper(self, audio_file):
        transcription = self.client.audio.transcriptions.create(
                                model="whisper-1",
                                file=audio_file,
                            )
        return transcription


# Funcao para refinar o texto da transcricao 
class OpenAICorrector():
    def __init__(self):
        self.client = OpenAI(api_key=os.environ.get('OPENAI_KEY'))

    def get_corrected_transcript(self, system_prompt, content):
        response = self.client.chat.completions.create(
            model="gpt-4o",
            temperature=0,
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": content
                }
            ]
        )
        return response.choices[0].message.content

# Funcao para diarizacao
class OpenAIDiarization():
    def __init__(self):
        self.client = OpenAI(api_key=os.environ.get('OPENAI_KEY'))

    def get_completion_diarization(self, prompt_given):
        messages = [{"role": "user", "content": prompt_given}]
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=1
        )
        return response.choices[0].message.content
    
# Funcao para a diarizacao do texto fragmentado
class OpenAIDiarization2():
    def __init__(self):
        self.client = OpenAI(api_key=os.environ.get('OPENAI_KEY'))

    def get_completion_diarization(self, prompt_given):
        prompt = prompt_given 
        messages = [{"role": "user", "content": prompt}]
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=1
        )
        return response.choices[0].message.content
    
# Chamando todas as funcoes definidas acima
openaiWhisper = OpenAIWhisper()
openaiCorrector = OpenAICorrector()
openaiDiarization = OpenAIDiarization()
openaiDiarization2 = OpenAIDiarization2()

## Diarização

In [54]:
# Questao da transcricao
# Abrindo o arquivo e chamando a funcao de transcricao
with open(output_audio_path, "rb") as audio_file:
    transcription = openaiWhisper.get_completion_whisper(audio_file=audio_file)
print(transcription.text)
# Obtendo o prompt para refinar o texto e chamando a funcao
prompt = f"""
Você é um taquígrafo de uma sessão na câmara dos deputados. / 
Sua tarefa é refinar o texto de uma transcrição de audio. / 
Essa transcrição consiste de um tribunal de justiça,
que possuí como objetivo a discussão e deliberação do Poder Legislativo 
sobre a elaboração de leis, a fiscalização dos atos do Governo e 
ainda sobre as manifestações das diversas opiniões e posições partidárias e da sociedade. / 
Apenas retorne o texto refinado, sem nenhuma divisão por orador ou algo do tipo. / 
Arrume palavras que parecem estranhas, não mude a transcrição em si. / 
Se há um erro de português, tente corrigi-lo. / 
Se há uma palavra muito estranha colocada na frase, troque-a.
"""
content = openaiCorrector.get_corrected_transcript(system_prompt=prompt, content=transcription.text)

# Questao da diarizacao

# Exemplo da diarizacao
example = f"""
Orador 1 (Presidente) : Boa tarde! / 
Orador 1 (Presidente) : Esta sessão está sendo gravada. / 
Orador 2 (Deputado) : Boa tarde a todos os presentes. / 
Orador 1 (Presidente) : Vamos começar com a discussão do projeto de lei número 123/2024. / 
Orador 3 (Deputado) : Senhor Presidente, solicito a palavra. / 
Orador 1 (Presidente) : Com a palavra o Deputado João. / 
Orador 3 (Deputado) : Agradeço, senhor Presidente. Venho aqui defender a importância deste projeto para nossa comunidade. / 
Orador 4 (Deputada) : Senhor Presidente, peço a palavra em aparte. / 
Orador 1 (Presidente) : Com a palavra a Deputada Maria. / 
Orador 4 (Deputada) : Gostaria de acrescentar que esse projeto é de extrema urgência para os nossos cidadãos. / 
Orador 1 (Presidente) : Obrigado pelos seus comentários. / 
Orador 5 (Deputado) : Senhor Presidente, solicito a palavra para uma questão de ordem. / 
Orador 1 (Presidente) : Pois não, Deputado Pedro. / 
Orador 5 (Deputado) : Sugiro que adiemos a votação deste projeto para podermos analisar melhor suas implicações. / 
Orador 1 (Presidente) : Vou considerar sua sugestão, Deputado. / 
Orador 1 (Presidente) : Mais algum deputado deseja se manifestar antes de prosseguirmos? / 
Orador 6 (Deputado) : Eu, senhor Presidente. / 
Orador 1 (Presidente) : Com a palavra o Deputado Carlos. / 
Orador 6 (Deputado) : Peço a palavra para destacar os pontos positivos deste projeto. / 
Orador 1 (Presidente) : Muito bem, Deputado Carlos. / 
Orador 1 (Presidente) : Se não houver mais inscrições para falar, vamos encerrar esta discussão e proceder à votação. / 
Orador 1 (Presidente) : A votação está aberta. / 
Orador 1 (Presidente) : Vou encerrar a sessão.
"""

prompt = f"""
Você é um taquígrafo de uma sessão na câmara dos deputados. / 
Sua tarefa é identificar o orador a partir de um texto transcrito. / 
Dado a transcrição delimitada por ```, entenda cada frase inteira e tente atribui-la a uma pessoa. / 
Gere cada trecho do texto com seu determinado rótulo para cada pessoa. / 
Normalmente, diversas pessoas participam de cada techo de sessão. / 
Deles, normalmente há a pessoa que é o presidente e os diversos deputados. / 
Você irá receber as instruções dos presidentes e dos deputados. / 

Presidente:
O papel do presidente é conduzir a sessão, isto é, chamar os deputados para falar,
enquanto que dos outros deputados é de discutir e deliberar sobre as leis. /
Quem vota não é o presidente, mas sim os deputados. /
Assim, o presidente normalmente é aquele que chama os deputados. /
Ele também pergunta aos partidos leis e afins, além de dar comunicados do plenário./
O presidente chama os partidos, da ordens e informa sobre a sessão. Ele também
fala de assuntos relacionados à sessão, como comunicados, ordens e fala sobre votos. /
Quem fala de características do plenário, como discussões, votos, características da sessão e qualquer coisa relacionada é o presidente. /
O presidente fala dos deputados e deputadas. Ele comunica parabenizações e premiações. /
O presidente é quem comenta, em grande parte das vezes, dos Deputados. / 
O presidente indica os votos da sessão. Dessa forma, ele indica se os votos são "Sim" ou "Não". /

Deputados:
Deputados, as vezes, vão se atravessar e falar fora da ordem. Dessa forma,
é importante que você consiga identificar quem é quem. /
Quem comenta em nome de algum partido ou bloco é sempre o deputado, nunca o presidente. /
Dessa forma, se alguém fala "Bloco do PSD, o voto é não" provavelmente é um deputado. /
Os Blocos e Partidos são: Governo, Maioria, Oposição, Minoria, UNIÃO, PP, PSD, MDB, REPUBLICANOS, /
PDT, PSDB, PODE, AVANTE, SOLIDARIEDADE e PRD. /
Quando alguem comenta "Presidente", provavelmente está se referindo ao presidente da sessão e não é a pessoa que conduz a sessão. /
Normalmente, quando um deputado fala e já trocaram de partido, ele não irá mais falar. /
O deputado realmente não irá mais falar se agradeceu pelo seu tempo ou algo nesse sentido. /
Dessa forma, algum orador que já foi classificado provavelmente não falará novamente após o presidente chamar outro partido. /
Normalmente, algum deputado que agradece, ou "encerra seu discurso" não falará mais. /
Partidos diferentes não irão ser representados por pessoas diferentes. /
Existem representantes diferentes de partidos semelhantes. Assim, dois deputados podem representar o mesmo partido ou bloco. /
Se há duas vezes uma resposta do mesmo partido ou bloco, são pessoas diferentes. /

Conclusão:
Quem faz comunicados é o presidente. Pense em todos tipos de comunicado, como informativos, lembranças, indicações. /
Busque entender o texto e identificar perguntas e respostas, que normalmente são respondidas por pessoas diferentes. / 
Busque palavras que concluam a frase de alguém, como agradecimentos. Após isso, a mesma pessoa provavelmente não irá mais falar. /
Nem sempre será o presidente que irá comunicar algo. Se alguém começa a falar, essa pessoa provavelmente irá continuar com seu discurso. /
Duas pessoas diferentes podem representar o mesmo partido ou bloco. /
Quem pede para complementar algo é sempre uma pessoa diferente. /
Esse texto está organizado por tempo, então classifique desde a primeira frase em ordem linear. / 
Retorne apenas o texto com cada um dos oradores. / 
Retorne o texto completo dividido por oradores. / 
Não tente inventar pessoas dentro do espaço. Pense em como seria a conversa e faça a identificação. / 
Retorne apenas o texto com a identificação de orador, sem nenhuma outra frase ou comentário. / 
Retorne apenas, por exemplo, 'Orador 1', 'Orador 2', etc. /
Não retorne o cargo, apenas o número do orador. / 
Se baseie no exemplo que será disponibilizado por ```.
Exemplo: ```{example}```
Transcrição: ```{content}```
"""

# Chamando a diarizacao
response = openaiDiarization.get_completion_diarization(prompt_given = prompt)

# Juntando em um json
transcription_dict = {
    'transcription (2.2)': content,#.strip(),
    'diarization (2.3)': response#.strip()
}
transcription_json = json.dumps(transcription_dict, ensure_ascii=False, indent=2)

# Retornando a resposta da transcricao
transcriptions = json.loads(transcription_json)
# Acessando as transcricoes
transcription = transcriptions["transcription (2.2)"]
print("Transcrição:", transcription)

# Retornando a resposta da diarizacao
diarization = json.loads(transcription_json)
# Acessando a diarizacao
diarization = diarization["diarization (2.3)"]
print("Diarização:", diarization)

não contam com o consenso dessa casa. Quanto nós ainda precisamos evoluir como seres humanos para entender, compreender a história universal, para saber que o que aconteceu durante a 2ª Guerra Mundial sobre o nazismo jamais pode se repetir no mundo e muito menos pode ser relativizado como foi pelo presidente Lula nessa última manifestação sua. É por isso que mais de 120 deputados federais já assinaram o pedido de impeachment que estará depois protocolado para a despache da presidência dessa casa e nós incluímos também uma denúncia à Procuradoria Geral da República contra Lula por racismo e antissemitismo. Chega de tolerar aquilo que não se pode tolerar. Vale para este projeto de lei, vale também para as manifestações absurdas de Lula. Para discutir a favor, deputada Laura Carneiro. Como eu estou escrita para encaminhar, eu abro mão da discussão. Deputado Cabo Gilberto, para discutir a matéria. Eu faço só o lembrete aos senhores e senhoras deputados que nós estamos em discussão da matér

## Ajustando a Transcrição - Sentença

In [168]:
transcricao_real = open(path + join + "/transcricao_real.txt", "r", encoding="utf8").read()
resultado = open(path + join + "/resultado.txt", "r", encoding="utf8").read()

In [169]:
sentencas = transcricao_real.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
treal = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(treal)

      orador                                           sentenca
0   Orador 2  Parlamento, a nossa casa e Vossa Excelência qu...
1   Orador 2                  Muito obrigada pela participação.
2   Orador 1  Deputada Antônia, eu quero parabenizar os visi...
3   Orador 1  Deputado, Vossa Excelência pode formular a que...
4   Orador 3  Baseado na resolução 19 de 2021, artigo 4, Sr....
5   Orador 3  Olha, não sei, Vossa Excelência é uma pessoa g...
6   Orador 3  Eu quero até fazer aqui uma homenagem a Vossa ...
7   Orador 3  Portanto, a Vossa Excelência, deputado Gilbert...
8   Orador 3  Vossa Excelência tem conduzido muito bem as qu...
9   Orador 3  Eu quero dizer a Vossa Excelência que, indepen...
10  Orador 3  Eu sou muito rápido em me inscrever, às vezes ...
11  Orador 3  Às vezes eu fico em 15º, 20º, às vezes eu não ...
12  Orador 3  Porém, hoje, Gustavo Gayer, deputado, estava n...
13  Orador 3     Aconteceu hoje algo diferente dos outros dias?
14  Orador 3                         Não

In [170]:
sentencas = resultado.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
result = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(result)

      orador                                           sentenca
0   Orador 2  Parlamento, a nossa casa e Vossa Excelência qu...
1   Orador 2                  Muito obrigada pela participação.
2   Orador 1  Deputada Antônia, eu quero parabenizar os visi...
3   Orador 1  Deputado, Vossa Excelência pode formular a que...
4   Orador 3  Baseado na resolução 19 de 2021, artigo 4, Sr....
5   Orador 3  Olha, não sei, Vossa Excelência é uma pessoa g...
6   Orador 3  Eu quero até fazer aqui uma homenagem a Vossa ...
7   Orador 3  Portanto, a Vossa Excelência, deputado Gilbert...
8   Orador 3  Vossa Excelência tem conduzido muito bem as qu...
9   Orador 3  Eu quero dizer a Vossa Excelência que, indepen...
10  Orador 3  Eu sou muito rápido em me inscrever, às vezes ...
11  Orador 3  Às vezes eu fico em 15º, 20º, às vezes eu não ...
12  Orador 3  Porém, hoje, Gustavo Gayer, deputado, estava n...
13  Orador 3     Aconteceu hoje algo diferente dos outros dias?
14  Orador 3                         Não

In [171]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import pandas as pd


result_sentences = result['sentenca'].tolist()
real_sentences = treal['sentenca'].tolist()

# Calculate accuracy
accuracy = accuracy_score(result['orador'], treal['orador'])

# Calculate precision and recall for each class
precision = precision_score(result['orador'], treal['orador'], average='weighted')
recall = recall_score(result['orador'], treal['orador'], average='weighted')
f1 = f1_score(result['orador'], treal['orador'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


## Ajustando a Transcrição - Palavra

In [208]:
transcricao_real = open(path + join + "/transcricao_real.txt", "r", encoding="utf8").read()
resultado = open(path + join + "/resultado.txt", "r", encoding="utf8").read()

In [209]:
sentencas = transcricao_real.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r' ')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
treal = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(treal)

       orador      sentenca
0    Orador 2   Parlamento,
1    Orador 2             a
2    Orador 2         nossa
3    Orador 2          casa
4    Orador 2             e
..        ...           ...
580  Orador 3            do
581  Orador 3          dia,
582  Orador 3           que
583  Orador 3           por
584  Orador 3  gentileza...

[585 rows x 2 columns]


In [210]:
sentencas = resultado.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r' ')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
result = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(result)

       orador      sentenca
0    Orador 2   Parlamento,
1    Orador 2             a
2    Orador 2         nossa
3    Orador 2          casa
4    Orador 2             e
..        ...           ...
580  Orador 3            do
581  Orador 3          dia,
582  Orador 3           que
583  Orador 3           por
584  Orador 3  gentileza...

[585 rows x 2 columns]


In [211]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import pandas as pd


result_sentences = result['sentenca'].tolist()
real_sentences = treal['sentenca'].tolist()

# Calculate accuracy
accuracy = accuracy_score(result['orador'], treal['orador'])

# Calculate precision and recall for each class
precision = precision_score(result['orador'], treal['orador'], average='weighted')
recall = recall_score(result['orador'], treal['orador'], average='weighted')
f1 = f1_score(result['orador'], treal['orador'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


# Diarização AssemblyAI

In [103]:
# Start by making sure the `assemblyai` package is installed.
# If not, you can install it by running the following command:
# pip install -U assemblyai
#
# Note: Some macOS users may need to use `pip3` instead of `pip`.

import assemblyai as aai

# Replace with your API key
aai.settings.api_key = 123 ##adicionar a key

# URL of the file to transcribe
FILE_URL = output_audio_path

config = aai.TranscriptionConfig(speaker_labels=True,
                                 language_code="pt")

transcriber = aai.Transcriber()
transcript = transcriber.transcribe(
  FILE_URL,
  config=config
)

for utterance in transcript.utterances:
  print(f"Speaker {utterance.speaker}: {utterance.text}")

Speaker A: Esse parlamento, a nossa casa e vossa excelência que preside essa sessão dessa tarde. Muito obrigada pela participação.
Speaker B: Deputada Antônia, eu quero parabenizar os visitantes e parabenizá-los por ter uma tão abençoada representante nessa casa como a deputada Antônia Lúcia, ok? Deputado, V. Exª pode formular a questão de ordem, por favor, colocando antes o artigo.
Speaker C: Baseado na resolução 19 de 2021, artigo 4, Sr. Presidente, V. Exª, aliás, eu quero fazer um elogio a V. Exª, Deputado Gilberto Nascimento. Olha, não sei V. Exª é uma pessoa generosa, afável, conduz muito bem as sessões, inclusive os breves comunicados, e agora também na fase da ordem do dia. Eu quero até fazer aqui uma homenagem à V. Exª, que pacientemente ouve a todos, organiza os trabalhos e consegue fazer com que esta casa que tem oposição e governo e muitos independentes se entendam nos momentos mais difíceis. Portanto, a V. Exª, deputado Gilberto Nascimento, a minha homenagem. V. Exª tem con

## Ajustando a Transcrição - Sentença

In [104]:
path = os.path.abspath("")
path = path.replace("\\", "/")
transcricao_real = open(path + "/assembly/dataset8/real_transcription.txt", "r", encoding="utf8").read()
resultado = open(path + "/assembly/dataset8/predicted_transcription.txt", "r", encoding="utf8").read()

In [105]:
sentencas = transcricao_real.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
treal = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(treal)

      orador                                           sentenca
0   Orador 1  Esse parlamento, a nossa casa e vossa excelênc...
1   Orador 1                  Muito obrigada pela participação.
2   Orador 2  Deputada Antônia, eu quero parabenizar os visi...
3   Orador 2                                       Deputado, V.
4   Orador 2  Exª pode formular a questão de ordem, por favo...
5   Orador 3  Baseado na resolução 19 de 2021, artigo 4, Sr....
6   Orador 3          Exª, aliás, eu quero fazer um elogio a V.
7   Orador 3                 Exª, Deputado Gilberto Nascimento.
8   Orador 3                                   Olha, não sei V.
9   Orador 3  Exª é uma pessoa generosa, afável, conduz muit...
10  Orador 3         Eu quero até fazer aqui uma homenagem à V.
11  Orador 3  Exª, que pacientemente ouve a todos, organiza ...
12  Orador 3                                     Portanto, a V.
13  Orador 3  Exª, deputado Gilberto Nascimento, a minha hom...
14  Orador 3                            

In [106]:
sentencas = resultado.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
result = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(result)

      orador                                           sentenca
0   Orador 1  Esse parlamento, a nossa casa e vossa excelênc...
1   Orador 1                  Muito obrigada pela participação.
2   Orador 2  Deputada Antônia, eu quero parabenizar os visi...
3   Orador 2                                       Deputado, V.
4   Orador 2  Exª pode formular a questão de ordem, por favo...
5   Orador 3  Baseado na resolução 19 de 2021, artigo 4, Sr....
6   Orador 3          Exª, aliás, eu quero fazer um elogio a V.
7   Orador 3                 Exª, Deputado Gilberto Nascimento.
8   Orador 3                                   Olha, não sei V.
9   Orador 3  Exª é uma pessoa generosa, afável, conduz muit...
10  Orador 3         Eu quero até fazer aqui uma homenagem à V.
11  Orador 3  Exª, que pacientemente ouve a todos, organiza ...
12  Orador 3                                     Portanto, a V.
13  Orador 3  Exª, deputado Gilberto Nascimento, a minha hom...
14  Orador 3                            

In [107]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import pandas as pd


result_sentences = result['sentenca'].tolist()
real_sentences = treal['sentenca'].tolist()

# Calculate accuracy
accuracy = accuracy_score(result['orador'], treal['orador'])

# Calculate precision and recall for each class
precision = precision_score(result['orador'], treal['orador'], average='weighted')
recall = recall_score(result['orador'], treal['orador'], average='weighted')
f1 = f1_score(result['orador'], treal['orador'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


Rodando o código base para diarização da AssemblyAI

## Ajustando a Transcrição - Palavra

In [97]:
path = os.path.abspath("")
path = path.replace("\\", "/")
transcricao_real = open(path + "/assembly/dataset7/real_transcription.txt", "r", encoding="utf8").read()
resultado = open(path + "/assembly/dataset7/predicted_transcription.txt", "r", encoding="utf8").read()

In [98]:
sentencas = transcricao_real.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r' ')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
treal = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(treal)

       orador       sentenca
0    Orador 1  Especialmente
1    Orador 1             os
2    Orador 1   empresários,
3    Orador 1        aqueles
4    Orador 1            que
..        ...            ...
427  Orador 4            com
428  Orador 4           esse
429  Orador 4   entendimento
430  Orador 4         porque
431  Orador 4           ele.

[432 rows x 2 columns]


In [99]:
sentencas = resultado.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r' ')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
result = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(result)

       orador       sentenca
0    Orador 1  Especialmente
1    Orador 1             os
2    Orador 1   empresários,
3    Orador 1        aqueles
4    Orador 1            que
..        ...            ...
427  Orador 4            com
428  Orador 4           esse
429  Orador 4   entendimento
430  Orador 4         porque
431  Orador 4           ele.

[432 rows x 2 columns]


In [100]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import pandas as pd


result_sentences = result['sentenca'].tolist()
real_sentences = treal['sentenca'].tolist()

# Calculate accuracy
accuracy = accuracy_score(result['orador'], treal['orador'])

# Calculate precision and recall for each class
precision = precision_score(result['orador'], treal['orador'], average='weighted')
recall = recall_score(result['orador'], treal['orador'], average='weighted')
f1 = f1_score(result['orador'], treal['orador'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.9652777777777778
Precision: 0.9756235827664399
Recall: 0.9652777777777778
F1 Score: 0.96919810468969


# Oracle

In [112]:
path = os.path.abspath("")

path = path.replace("\\", "/")
print(path)

c:/Users/anton/Videos/boll/boll/diarizacao_artigo/diarizacao


In [191]:
# Carregar o arquivo JSON
with open('oracle/dataset8/oracle.json', 'r', encoding='utf-8') as file:
    dados_json = json.load(file)

# Extrair tokens
tokens = dados_json['transcriptions'][0]['tokens']

# Lista para armazenar as falas formatadas na ordem correta
formatted_falas = []

# Variável para armazenar a fala atual e o índice do orador atual
current_speech = []
current_speaker = None

# Iterar sobre os tokens para manter a linha do tempo
for token in tokens:
    orador = token['speakerIndex']
    palavra = token['token']

    if orador != current_speaker:
        # Se o orador mudou, salvar a fala anterior
        if current_speech:
            formatted_falas.append(f"Orador {current_speaker}: {' '.join(current_speech)} /")
        # Resetar a fala atual e atualizar o orador
        current_speech = [palavra]
        current_speaker = orador
    else:
        # Adicionar a palavra à fala atual
        current_speech.append(palavra)

# Adicionar a última fala ao final da iteração
if current_speech:
    formatted_falas.append(f"Orador {current_speaker}: {' '.join(current_speech)} /")

# Unir todas as falas formatadas em uma única string
dados_formatados = '\n'.join(formatted_falas)

print(dados_formatados)

Orador 0: parlamento, a nossa casa e vossa excelência que preside esta sessão nesta tarde. Muito obrigada pela participação. Ok, deputada Antônia, eu quero parabenizar os visitantes e parabenizar-os por ter uma tão abençoada representante nesta casa como a deputada Antônia Lúcia, ok? Deputado, a vossa excelência pode formular a questão de ordem, por favor, colocando antes o artigo. Baseado na Resolução 19 de 2021, o artigo 4º, seu presidente. Vossa excelência, aliás, eu quero fazer um elogio à vossa excelência, deputado Gilberto Nascimento. Olha, não sei... Vossa excelência é uma pessoa generosa, A Fável conduz muito bem as sessões, inclusive os breves comunicados e agora também na fase da Hora do Dia. Eu quero até fazer aqui uma homenagem à Vossa Excellência que, pacientemente, ouve a todos, organiza os trabalhos e consegue fazer com que essa casa que tem oposição e governo e muitos independentes se entendam nos momentos mais difíceis. Portanto, a vossa excelência, deputado Gilberto N

## Ajustando a Transcrição - Sentença

In [192]:
transcricao_real = open(path + "/oracle/dataset8/transcricao_real.txt", "r", encoding="utf8").read()
resultado = open(path + "/oracle/dataset8/transcricao_predita.txt", "r", encoding="utf8").read()

In [195]:
sentencas = transcricao_real.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
treal = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(treal)

      orador                                           sentenca
0   Orador 0  parlamento, a nossa casa e vossa excelência qu...
1   Orador 0                  Muito obrigada pela participação.
2   Orador 1  Ok, deputada Antônia, eu quero parabenizar os ...
3   Orador 1  Deputado, a vossa excelência pode formular a q...
4   Orador 2  Baseado na Resolução 19 de 2021, o artigo 4º, ...
5   Orador 2  Vossa excelência, aliás, eu quero fazer um elo...
6   Orador 2                                   Olha, não sei...
7   Orador 2  Vossa excelência é uma pessoa generosa, A Fáve...
8   Orador 2  Eu quero até fazer aqui uma homenagem à Vossa ...
9   Orador 2  Portanto, a vossa excelência, deputado Gilbert...
10  Orador 2  Vossa excelência tem conduzido muito bem as qu...
11  Orador 2  E por ser vossa excelência, muito justo no tra...
12  Orador 2  Eu quero dizer à vossa excelência que, indepen...
13  Orador 2  Eu sou muito rápido em me inscrever, às vezes ...
14  Orador 2  Às vezes eu fico em 15, 20

In [196]:
sentencas = resultado.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
result = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(result)

      orador                                           sentenca
0   Orador 0  parlamento, a nossa casa e vossa excelência qu...
1   Orador 0                  Muito obrigada pela participação.
2   Orador 0  Ok, deputada Antônia, eu quero parabenizar os ...
3   Orador 0  Deputado, a vossa excelência pode formular a q...
4   Orador 0  Baseado na Resolução 19 de 2021, o artigo 4º, ...
5   Orador 0  Vossa excelência, aliás, eu quero fazer um elo...
6   Orador 0                                   Olha, não sei...
7   Orador 0  Vossa excelência é uma pessoa generosa, A Fáve...
8   Orador 0  Eu quero até fazer aqui uma homenagem à Vossa ...
9   Orador 0  Portanto, a vossa excelência, deputado Gilbert...
10  Orador 0  Vossa excelência tem conduzido muito bem as qu...
11  Orador 0  E por ser vossa excelência, muito justo no tra...
12  Orador 0  Eu quero dizer à vossa excelência que, indepen...
13  Orador 0  Eu sou muito rápido em me inscrever, às vezes ...
14  Orador 0  Às vezes eu fico em 15, 20

In [197]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import pandas as pd


result_sentences = result['sentenca'].tolist()
real_sentences = treal['sentenca'].tolist()

# Calculate accuracy
accuracy = accuracy_score(result['orador'], treal['orador'])

# Calculate precision and recall for each class
precision = precision_score(result['orador'], treal['orador'], average='weighted')
recall = recall_score(result['orador'], treal['orador'], average='weighted')
f1 = f1_score(result['orador'], treal['orador'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.06451612903225806
Precision: 1.0
Recall: 0.06451612903225806
F1 Score: 0.12121212121212122


## Ajustando a Transcrição - Palavra

In [198]:
transcricao_real = open(path + "/oracle/dataset8/transcricao_real.txt", "r", encoding="utf8").read()
resultado = open(path + "/oracle/dataset8/transcricao_predita.txt", "r", encoding="utf8").read()

In [199]:
sentencas = transcricao_real.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r' ')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
treal = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(treal)

       orador     sentenca
0    Orador 0  parlamento,
1    Orador 0            a
2    Orador 0        nossa
3    Orador 0         casa
4    Orador 0            e
..        ...          ...
581  Orador 2          que
582  Orador 2           se
583  Orador 2         abra
584  Orador 2            a
585  Orador 2        ordem

[586 rows x 2 columns]


In [200]:
sentencas = resultado.split(" / ")

# Inicializar listas para os oradores e as sentenças
oradores = []
frases = []

# Expressão regular para dividir as frases
import re
regex = re.compile(r' ')

# Processar cada sentença
for sentenca in sentencas:
    if ": " in sentenca:
        orador, texto = sentenca.split(": ", 1)
        # Dividir as frases pelo ponto final, exclamação ou interrogação
        frases_texto = regex.split(texto)
        for frase in frases_texto:
            frase = frase.strip()
            if frase:
                oradores.append(orador.strip())
                frases.append(frase)

# Criar o DataFrame
import pandas as pd
result = pd.DataFrame({
    'orador': oradores,
    'sentenca': frases
})

# Exibir o DataFrame
print(result)

       orador     sentenca
0    Orador 0  parlamento,
1    Orador 0            a
2    Orador 0        nossa
3    Orador 0         casa
4    Orador 0            e
..        ...          ...
581  Orador 0          que
582  Orador 0           se
583  Orador 0         abra
584  Orador 0            a
585  Orador 0        ordem

[586 rows x 2 columns]


In [201]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import pandas as pd


result_sentences = result['sentenca'].tolist()
real_sentences = treal['sentenca'].tolist()

# Calculate accuracy
accuracy = accuracy_score(result['orador'], treal['orador'])

# Calculate precision and recall for each class
precision = precision_score(result['orador'], treal['orador'], average='weighted')
recall = recall_score(result['orador'], treal['orador'], average='weighted')
f1 = f1_score(result['orador'], treal['orador'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.02901023890784983
Precision: 1.0
Recall: 0.02901023890784983
F1 Score: 0.05638474295190713
